In [ ]:
from mainnet_launch.data_fetching.get_state_by_block import (
    get_state_by_one_block,
    build_blocks_to_use,
    get_raw_state_by_blocks,
    safe_normalize_6_with_bool_success,
    safe_normalize_with_bool_success,
)

import os
import pandas as pd
from tqdm import tqdm
import concurrent.futures as cf

from mainnet_launch.database.postgres_operations import (
    get_full_table_as_df_with_block,
    get_full_table_as_df,
    get_full_table_as_df_with_tx_hash,
)
from mainnet_launch.database.schema.full import (
    DestinationStates,
    Destinations,
    AutopoolDestinations,
    RebalanceEvents,
    RebalancePlans,
    Blocks,
)
import pandas as pd

from multicall import Call
import numpy as np
import time
import datetime
from mainnet_launch.data_fetching.defi_llama.fetch_timestamp import fetch_blocks_by_unix_timestamps_defillama

from mainnet_launch.constants import (
    AUTO_USD,
    ETH_CHAIN,
    AutopoolConstants,
    AUTO_DOLA,
    BASE_USD,
    BASE_CHAIN,
    ALL_AUTOPOOLS,
    AUTO_ETH, BASE_ETH
)

import plotly.express as px
import plotly.io as pio

pio.templates.default = None


def _extract_limited_events_data(
    autopool: AutopoolConstants,
    events: pd.DataFrame,
    plans: pd.DataFrame,
    destination_states: pd.DataFrame,
    destinations: pd.DataFrame,
) -> pd.DataFrame:

    limited_events_df = events[
        ["destination_in", "destination_out", "block", "safe_value_out", "rebalance_file_path"]
    ].copy()

    get_fee_and_base_apr = destination_states.set_index(["destination_vault_address", "rebalance_plan_key"])[
        "fee_plus_base_apr"
    ].to_dict()


    limited_events_df["fee_and_base_out"] = limited_events_df.apply(
        lambda row: get_fee_and_base_apr.get((row["destination_out"], row["rebalance_file_path"]), None), axis=1
    )
    limited_events_df["fee_and_base_in"] = limited_events_df.apply(
        lambda row: get_fee_and_base_apr.get((row["destination_in"], row["rebalance_file_path"]), None), axis=1
    )

    destination_names = destinations.set_index("destination_vault_address")["underlying_name"].to_dict()
    exchange_names = destinations.set_index("destination_vault_address")["exchange_name"].to_dict()
    pool_addresses = destinations.set_index("destination_vault_address")["pool"].to_dict()

    limited_events_df["destination_in_name"] = limited_events_df["destination_in"].map(destination_names)
    limited_events_df["destination_out_name"] = limited_events_df["destination_out"].map(destination_names)
    limited_events_df["out_exchange_name"] = limited_events_df["destination_out"].map(exchange_names)
    limited_events_df["in_exchange_name"] = limited_events_df["destination_in"].map(exchange_names)
    limited_events_df["pool_in"] = limited_events_df["destination_in"].map(pool_addresses)
    limited_events_df["pool_out"] = limited_events_df["destination_out"].map(pool_addresses)

    # Join limited_events_df with plans on rebalance_file_path = file_name
    limited_events_df = limited_events_df.merge(plans, left_on="rebalance_file_path", right_on="file_name", how="left")

    return limited_events_df


def load_data(autopool: AutopoolConstants):
    destinations = get_full_table_as_df(Destinations, where_clause=Destinations.chain_id == autopool.chain.chain_id)
    autopool_destinations = get_full_table_as_df(
        AutopoolDestinations, where_clause=AutopoolDestinations.autopool_vault_address == autopool.autopool_eth_addr
    )
    # 2 min to fetch
    destination_states = get_full_table_as_df_with_block(
        DestinationStates,
        where_clause=DestinationStates.destination_vault_address.in_(
            destinations["destination_vault_address"].tolist()
        ),
    )
    # I don't think destination states is properly gettin gthe fee and base apr from the rebalance event
    plans = get_full_table_as_df(
        RebalancePlans, where_clause=RebalancePlans.autopool_vault_address == autopool.autopool_eth_addr
    )
    events = get_full_table_as_df_with_tx_hash(
        RebalanceEvents, where_clause=RebalanceEvents.autopool_vault_address == autopool.autopool_eth_addr
    )
    mainnet_blocks = get_full_table_as_df(Blocks, where_clause=Blocks.chain_id == autopool.chain.chain_id).sort_values(
        "block"
    )

    limited_events_df = _extract_limited_events_data(autopool, events, plans, destination_states, destinations)
    return destinations, autopool_destinations, destination_states, plans, events, mainnet_blocks, limited_events_df


VP_METHODS = [
    # --- common "rate / virtual price" style ---
    ("getRate", ["getRate()(uint256)"], None),
    ("rate", ["rate()(uint256)"], None),
    ("getExchangeRate", ["getExchangeRate()(uint256)"], None),
    ("get_virtual_price", ["get_virtual_price()(uint256)"], None),
    ("getVirtualPrice", ["getVirtualPrice()(uint256)"], None),

    # --- ERC-4626 / vault share->asset conversions (probe with 1e18 shares) ---
    ("convertToAssets_1e18", ["convertToAssets(uint256)(uint256)", int(10**18)], int(10**18)),
    ("previewRedeem_1e18", ["previewRedeem(uint256)(uint256)", int(10**18)], int(10**18)),
    ("totalAssets", ["totalAssets()(uint256)"], None),

    # --- Yearn / vault PPS variants ---
    ("pricePerShare", ["pricePerShare()(uint256)"], None),
    ("getPricePerFullShare", ["getPricePerFullShare()(uint256)"], None),

    # --- Compound-style cTokens ---
    ("exchangeRateStored", ["exchangeRateStored()(uint256)"], None),

    # --- Lido / LSD wrappers ---
    ("stEthPerToken", ["stEthPerToken()(uint256)"], None),
]


def build_vp_calls(pool_address: str):
    calls = []
    for suffix, fn, _ in VP_METHODS:
        key = f"{pool_address}:{suffix}"
        calls.append(
            Call(
                target=pool_address,
                function=fn,
                returns=[(key, safe_normalize_with_bool_success)],
            )
        )
    return calls


def _get_working_virtual_price_column(df: pd.DataFrame, cols_in_priority: list[str]) -> pd.Series:
    for col in cols_in_priority:
        if not any(df[col].isna()):
            return df[col]
    print(df[cols_in_priority])

    raise ValueError("could not identify working virtual price column")


def compute_apr(vp_df: pd.DataFrame) -> pd.DataFrame:

    t0 = vp_df.index[0]
    days = (vp_df.index - t0).total_seconds() / 86400.0

    out0 = vp_df["out_vp"].iloc[0]
    in0 = vp_df["in_vp"].iloc[0]

    # annualized % using actual elapsed days; guard day=0 at start
    vp_df["out_ann_pct"] = np.where(days > 0, ((vp_df["out_vp"] / out0) ** (365.0 / days) - 1.0), np.nan)
    vp_df["in_ann_pct"] = np.where(days > 0, ((vp_df["in_vp"] / in0) ** (365.0 / days) - 1.0), np.nan)

    return vp_df[["block", "out_vp", "in_vp", "out_ann_pct", "in_ann_pct"]]


def _fetch_vp_df(blocks_to_query: list[int], row: pd.Series, autopool: AutopoolConstants) -> pd.DataFrame:
    out_addr = row["pool_out"]
    in_addr = row["pool_in"]

    calls_to_make = []
    calls_to_make += build_vp_calls(out_addr)
    calls_to_make += build_vp_calls(in_addr)

    vp_df = get_raw_state_by_blocks(
        calls_to_make,
        blocks_to_query,
        autopool.chain,
        include_block_number=True,
    )

    # Coalesce per destination in the same priority order as VP_METHODS
    out_cols = [f"{out_addr}:{suffix}" for suffix, _, _ in VP_METHODS]
    in_cols = [f"{in_addr}:{suffix}" for suffix, _, _ in VP_METHODS]

    vp_df["out_vp"] = _get_working_virtual_price_column(vp_df, out_cols)
    vp_df["in_vp"] = _get_working_virtual_price_column(vp_df, in_cols)

    apr_df = compute_apr(vp_df)
    return apr_df


def determine_forward_looking_vp(autopool: AutopoolConstants, row: pd.Series):
    try:
        start_block = int(row["block"])
        chain_to_approx_blocks_per_day = {
            ETH_CHAIN: 7150,  # crude approx
            BASE_CHAIN: 43200,  # crude approx
        }
        approx_blocks_per_day = chain_to_approx_blocks_per_day[autopool.chain]

        block_30_days = start_block + (approx_blocks_per_day * 30)
        block_60_days = start_block + (approx_blocks_per_day * 60)

        today_block = autopool.chain.get_block_near_top()

        if (block_30_days > today_block) or (block_60_days > today_block):
            return {
                **row,
                "valid": False,
            }

        blocks_to_query = [start_block, block_30_days, block_60_days]

        apr_df = _fetch_vp_df(blocks_to_query, row, autopool)

        actual_30_day_fee_and_base_out = apr_df.loc[apr_df["block"] == block_30_days, "out_ann_pct"].values[0]
        actual_60_day_fee_and_base_out = apr_df.loc[apr_df["block"] == block_60_days, "out_ann_pct"].values[0]

        actual_30_day_fee_and_base_in = apr_df.loc[apr_df["block"] == block_30_days, "in_ann_pct"].values[0]
        actual_60_day_fee_and_base_in = apr_df.loc[apr_df["block"] == block_60_days, "in_ann_pct"].values[0]

        block_timstamp_30_days = apr_df["block"].loc[apr_df["block"] == block_30_days].index[0]
        block_timstamp_60_days = apr_df["block"].loc[apr_df["block"] == block_60_days].index[0]

        return {
            **row,
            "actual_30_day_fee_and_base_out": actual_30_day_fee_and_base_out,
            "actual_60_day_fee_and_base_out": actual_60_day_fee_and_base_out,
            "actual_30_day_fee_and_base_in": actual_30_day_fee_and_base_in,
            "actual_60_day_fee_and_base_in": actual_60_day_fee_and_base_in,
            "start_block": start_block,
            "block_30_days": block_30_days,
            "block_60_days": block_60_days,
            "timestamp_30_days": block_timstamp_30_days,
            "timestamp_60_days": block_timstamp_60_days,
            "valid": True,
        }
    except Exception as e:
        print(f"Error processing row with block {row['block']}: {e}")
        return {
            **row,
            "valid": False,
            "error": str(e),
            "error_type": type(e).__name__,
        }


def add_virtual_price_values_no_threads(autopool: AutopoolConstants, limited_events_df: pd.DataFrame) -> pd.DataFrame:
    new_rows = limited_events_df.apply(lambda row: determine_forward_looking_vp(autopool, row), axis=1).tolist()
    all_results_df = pd.DataFrame.from_records(new_rows)

    all_results_df.loc[
        all_results_df["destination_out_x"] == autopool.autopool_eth_addr,
        ["actual_30_day_fee_and_base_out", "actual_60_day_fee_and_base_out"],
    ] = 0
    all_results_df.loc[
        all_results_df["destination_in_x"] == autopool.autopool_eth_addr,
        ["actual_30_day_fee_and_base_in", "actual_60_day_fee_and_base_in"],
    ] = 0

    return all_results_df



autopool = AUTO_ETH


destinations, autopool_destinations, destination_states, plans, events, mainnet_blocks, limited_events_df = (
    load_data(autopool)
)
# full_df = add_virtual_price_values_no_threads(autopool, limited_events_df.reset_index())


# for autopool in ALL_AUTOPOOLS:
#     try:
#         data_dir = f"rebalance_analysis"
#         os.makedirs(data_dir, exist_ok=True)
#         destinations, autopool_destinations, destination_states, plans, events, mainnet_blocks, limited_events_df = (
#             load_data(autopool)
#         )
#         full_df = add_virtual_price_values_no_threads(autopool, limited_events_df.reset_index())
#         df = full_df[full_df["valid"] == True]
#         filename = os.path.join(data_dir, f"{autopool.name}_fee_and_base_apr.csv")
#         full_df.to_csv(filename)
#         # print(full_df["valid"].value_counts())
#         print(f"Loaded {full_df.shape} rebalance events for {autopool.name}")
#     except Exception as e:
#         print(f"Error processing autopool {autopool.name}: {e}")

In [6]:
destination_states

,destination_vault_address,block,chain_id,incentive_apr,fee_apr,base_apr,points_apr,fee_plus_base_apr,total_apr_in,total_apr_out,underlying_token_total_supply,safe_total_supply,lp_token_spot_price,lp_token_safe_price,from_rebalance_plan,rebalance_plan_timestamp,rebalance_plan_key
datetime,,,,,,,,,,,,,,,,,
2026-02-07 03:39:47+00:00,0x232d4755B2d88895472AfBD7F0C6D0acb3b65D04,24402316,1,0.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,9.451634e+10,NaN,1.106086,1.105968,True,1.770436e+09,rebalance_plan_1770435577_0x408b6A3E2Daf288864...
2026-02-07 03:39:47+00:00,0x227f68BD50Ad87E33728dBCb79Cb843eEcB34dcd,24402316,1,0.0,NaN,NaN,NaN,0.047162,0.047162,0.047162,1.009992e+13,NaN,1.011715,1.011607,True,1.770436e+09,rebalance_plan_1770435577_0x408b6A3E2Daf288864...
2026-02-07 03:39:47+00:00,0x408b6A3E2Daf288864968454AAe786a2A042Df36,24402316,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.170972e+02,NaN,1.000000,1.000000,True,1.770436e+09,rebalance_plan_1770435577_0x408b6A3E2Daf288864...
2026-02-07 03:39:47+00:00,0x4b1eA6f0E97cB0e859b1521a4548FBd0a94e6a63,24402316,1,0.0,NaN,NaN,NaN,0.413238,0.413238,0.413238,1.317461e+10,NaN,1.039206,1.039095,True,1.770436e+09,rebalance_plan_1770435577_0x408b6A3E2Daf288864...
2026-02-07 03:39:47+00:00,0x614a7d3b4e287555B9D6050fC59655C51961112B,24402316,1,0.0,NaN,NaN,NaN,0.000000,0.000000,0.000000,1.345240e+07,NaN,1.014855,1.014747,True,1.770436e+09,rebalance_plan_1770435577_0x408b6A3E2Daf288864...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-10 21:31:11+00:00,0xFDa49984eb4eA4075b8b451032849347C633E94b,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.011900,1.012070,False,NaN,None
2024-09-10 21:31:11+00:00,0x5c6aeb9ef0d5BbA4E6691f381003503FD0D45126,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.024024,1.024000,False,NaN,None
2024-09-10 21:31:11+00:00,0x896eCc16Ab4AFfF6cE0765A5B924BaECd7Fa455a,20722910,1,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1.028141,1.028136,False,NaN,None
